In [88]:
import pandas as pd
from os import path
import gensim
from gensim.models import Phrases
from gensim.models.word2vec import LineSentence

import glob, os, re
import numpy as np
import pandas as pd

In [90]:
%%time
lyrics_data = 'songs_rawdata\트로트_4990곡_240404.csv'

text_filepath = lyrics_data.split('\\')[-1].split('.csv')[0] + '.txt'
bigram_filepath = lyrics_data.split('\\')[-1].split('.csv')[0] + '_bigram.txt'
bigram_model_filepath = lyrics_data.split('\\')[-1].split('_')[0] + '_bigram_model'
corpus_filepath = lyrics_data.split('\\')[-1].split('.csv')[0] + '_bigram_corpus.csv'

indi = pd.read_csv(lyrics_data)
with open(text_filepath, 'w', encoding = 'utf-8') as f:
    for lyrics in indi.Lyrics.values:
        if pd.isnull(lyrics):
            continue
        f.write(lyrics+'\n')

CPU times: total: 62.5 ms
Wall time: 84.1 ms


In [91]:
sentences_normalized_path = text_filepath
unigram_sentences = LineSentence(sentences_normalized_path)
bigram_model = Phrases(unigram_sentences)
bigram_model.save(bigram_model_filepath)

In [92]:
%%time
bigram_sentences_filepath = bigram_filepath
with open(bigram_sentences_filepath, 'w', encoding = 'utf-8') as f:
    for unigram_sentence in unigram_sentences:
        bigram_sentence = bigram_model[unigram_sentence]
        f.write(' '.join(bigram_sentence) + '\n')

CPU times: total: 1.61 s
Wall time: 1.94 s


In [93]:
txt_path = bigram_filepath
raw_corpus = []

with open(txt_path, 'r', encoding = 'utf-8') as f:
    raw = f.read().splitlines()
    raw_corpus.extend(raw)
print(len(raw_corpus))

201607


In [94]:
def preprocess_sentence(sentence):
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)
    sentence = re.sub(r"[^a-zA-Z가-힣?.!,¿,_]+", " ", sentence)
    sentence = sentence.strip()
    sentence = re.sub(r"\(.\)", " ", sentence) # 7
    sentence = '<start> ' + sentence + ' <end>'
    return sentence

In [95]:
corpus = []
for sentence in raw_corpus:
    if len(sentence) == 0:
        continue
    if sentence[-1] == ':':
        continue
    if len(sentence)>150:
        continue
    
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
    
print(len(corpus))
print(corpus[:10])

201549
['<start> I m hurting baby <end>', '<start> I m broken down <end>', '<start> I need your loving loving <end>', '<start> I need it now <end>', '<start> When I m without you <end>', '<start> I m something weak <end>', '<start> You got me begging begging <end>', '<start> I m on my knees <end>', '<start> I don t wanna be needing your love <end>', '<start> I just wanna be deep in your love <end>']


In [96]:
df = pd.DataFrame({'col':corpus})
df.to_csv(corpus_filepath)